# Imports

In [145]:
import pandas as pd
import numpy as np

import seaborn as sns

# Load Data

- Load Train
- Load Test
- Combine Train and Test

In [146]:
train = pd.read_csv('../../data/original_data/train.csv')
test = pd.read_csv('../../data/original_data/test.csv')

len_train = len(train)
len_test = len(test)

train.shape, test.shape

((8693, 14), (4277, 13))

In [147]:
df = pd.concat([train, test])

## Confirm Concatenated Shape

In [148]:
df.shape[0] == test.shape[0] + train.shape[0]

True

In [149]:
df.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


# Feature Engineering

In [150]:
df['PassengerGroup'] = df['PassengerId'].apply(lambda x: x.split('_')[0])

group_size = df.groupby('PassengerGroup').size().reset_index(name='GroupSize')
df = df.merge(group_size, on='PassengerGroup')

df['Solo'] = df['GroupSize'].apply(lambda x: 1 if x == 1 else 0)

df['LastName'] = df['Name'].apply(lambda x: x.split(' ')[1] if not pd.isna(x) else np.nan)

# Null Values

## HomePlanet

- Check for passengers in same PassengerGroup
- Check for passengers with same surname
- Use mode to imput

In [151]:
display(f'The number of missing HomePlanet Values before is: {df['HomePlanet'].isna().sum()}')

df['HomePlanet'] = df.groupby('PassengerGroup')['HomePlanet'].transform(lambda x: x.fillna(x.mode()[0] if not x.mode().empty else np.nan))

display(f'The number of missing HomePlanet Values after is: {df['HomePlanet'].isna().sum()}')

'The number of missing HomePlanet Values before is: 288'

/tmp/ipykernel_34521/3920154015.py:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['HomePlanet'] = df.groupby('PassengerGroup')['HomePlanet'].transform(lambda x: x.fillna(x.mode()[0] if not x.mode().empty else np.nan))


'The number of missing HomePlanet Values after is: 157'

In [152]:
def impute_home_planet(row, df):
    if pd.isna(row['HomePlanet']):
        last_name = row['LastName']
        same_last_name = df[(df['LastName'] == last_name) & (df['HomePlanet'].notna())]
        if not same_last_name.empty:
            return same_last_name['HomePlanet'].values[0]
    return row['HomePlanet']



df['HomePlanet'] = df.apply(lambda row: impute_home_planet(row, df), axis=1)

display(f'The number of missing HomePlanet Values after is: {df['HomePlanet'].isna().sum()}')

'The number of missing HomePlanet Values after is: 13'

In [153]:
df['HomePlanet'] = df['HomePlanet'].fillna(df['HomePlanet'].mode()[0])
display(f'The number of missing HomePlanet Values after is: {df['HomePlanet'].isna().sum()}')

'The number of missing HomePlanet Values after is: 0'

## Destination

- Check for passengers in same PassengerGroup
- Check for passengers with same surname
- Use mode to imput

In [154]:
display(f'The number of missing Destination Values before is: {df['Destination'].isna().sum()}')

df['Destination'] = df.groupby('PassengerGroup')['Destination'].transform(lambda x: x.fillna(x.mode()[0] if not x.mode().empty else np.nan))

display(f'The number of missing Destination Values after is: {df['Destination'].isna().sum()}')

'The number of missing Destination Values before is: 274'

/tmp/ipykernel_34521/305662106.py:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Destination'] = df.groupby('PassengerGroup')['Destination'].transform(lambda x: x.fillna(x.mode()[0] if not x.mode().empty else np.nan))


'The number of missing Destination Values after is: 154'

In [155]:
def impute_destination(row, df):
    if pd.isna(row['Destination']):
        last_name = row['LastName']
        same_last_name = df[(df['LastName'] == last_name) & (df['Destination'].notna())]
        if not same_last_name.empty:
            return same_last_name['Destination'].values[0]
    return row['Destination']



df['Destination'] = df.apply(lambda row: impute_destination(row, df), axis=1)

display(f'The number of missing Destination Values after is: {df['Destination'].isna().sum()}')

'The number of missing Destination Values after is: 6'

In [156]:
df['Destination'] = df['Destination'].fillna(df['Destination'].mode()[0])
display(f'The number of missing Destination Values after is: {df['Destination'].isna().sum()}')

'The number of missing Destination Values after is: 0'

## VIP

In [157]:
df['TotalSpend'] = df[['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']].sum(axis=1)

In [158]:
df_vip = df[df['VIP'] == True]
df_no_vip = df[df['VIP'] == False]

df_vip.shape, df_no_vip.shape

((273, 19), (12401, 19))

In [159]:
f'The mean amount of money spent by VIPs was: {df_vip['TotalSpend'].mean()}'

'The mean amount of money spent by VIPs was: 4595.542124542125'

In [160]:
vip_null_mask = df['VIP'].isna()

df.loc[vip_null_mask, 'VIP'] = df.loc[vip_null_mask, 'TotalSpend'] >= 4000

In [161]:
df.head(4)

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,PassengerGroup,GroupSize,Solo,LastName,TotalSpend
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False,0001,1,1,Ofracculy,0.0
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True,0002,1,1,Vines,736.0
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False,0003,2,0,Susent,10383.0
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False,0003,2,0,Susent,5176.0


In [162]:
df.isna().sum()

PassengerId          0
HomePlanet           0
CryoSleep          310
Cabin              299
Destination          0
Age                270
VIP                  0
RoomService        263
FoodCourt          289
ShoppingMall       306
Spa                284
VRDeck             268
Name               294
Transported       4277
PassengerGroup       0
GroupSize            0
Solo                 0
LastName           294
TotalSpend           0
dtype: int64